<a href="https://colab.research.google.com/github/HENICHE-Thilleli/Time-Series-Forecasting-Using-Machine-Learning/blob/main/KNN/KNN_Date_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Charger les données
df = pd.read_csv('/content/drive/MyDrive/external.csv', parse_dates=True, sep=";")
df['date'] = pd.to_datetime(df['date'])

df1 = df[['date', 'PriceUSD']]
df1 = df1.set_index('date')

def create_features(df1):
    """
    Create time series features based on time series index.
    """
    df1 = df1.copy()
    df1['dayofweek'] = df1.index.dayofweek
    df1['quarter'] = df1.index.quarter
    df1['month'] = df1.index.month
    df1['year'] = df1.index.year
    df1['dayofyear'] = df1.index.dayofyear
    df1['dayofmonth'] = df1.index.day
    df1['weekofyear'] = df1.index.isocalendar().week
    return df1

df1 = create_features(df1)

In [ ]:
FEATURES = ['year', 'dayofweek', 'dayofmonth']
TARGET = ['PriceUSD']

train = df1[:int(len(df1)*0.70)]
test = df1[int(len(df1)*0.70):]

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

In [ ]:
# Entraînement et prédiction avec KNN
knn = KNeighborsRegressor(n_neighbors=2)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [ ]:
# Calcul des métriques d'évaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
bias = np.mean(y_pred - y_test.values.reshape(-1, 1))

print(f"Biais : {bias:.2f}")
print("MSE: ", mse)
print(f"R²: {r2:.4f}")

In [ ]:
# Tracer le graphique des scores R2 en fonction de K
r2_scores = []
for k in range(1, 21):
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
plt.plot(range(1, 21), r2_scores)
plt.xlabel('K')
plt.ylabel('R-squareD')
plt.title('R-squareD as a function of K')
#plt.savefig('/content/drive/MyDrive/Kfactor.png')
plt.show()

In [ ]:
dfplot=df[['date', 'PriceUSD']]
dfplot = dfplot.set_index('date')
dfplot.index = pd.to_datetime(dfplot.index)
train = dfplot[:int(len(df1)*0.70)]
test = dfplot[int(len(df1)*0.70):]
print(len(df1))
print(len(test))
print(len(train))
fig, ax = plt.subplots(figsize=(15, 5))
train.plot(ax=ax, label='Training Set', title='Data Train/Test Split')
test.plot(ax=ax, label='Test Set')
ax.axvline(df1.iloc[int(len(df1)*0.70),0], color='black', ls='--')
ax.legend(['Training Set', 'Test Set'])
plt.show()

In [ ]:
# Tracer le graphique des prédictions et des données réelles
test = df[int(len(df1)*0.70):]
dfplot = test[['date', 'PriceUSD']]
dfplot['prediction'] = y_pred
dfplot = dfplot.set_index('date')

fig, ax = plt.subplots(figsize=(15, 5))
dfplot.plot(ax=ax, title='Predictions/Real data')
ax.legend(['Real data', 'Predictions'])
#plt.savefig('/content/drive/MyDrive/KNNdate.png')
dfplot.to_csv('/content/drive/MyDrive/KNN.csv')
plt.show()